In [ ]:
%run Global.ipynb
%run Helper_Functions.ipynb

In [ ]:
train_df_ohe_selected = pd.read_csv(DATASET_DIR + "/train_df_ohe_selected.csv", keep_default_na=False)
train_selected_df, val_selected_df = train_test_split(train_df_ohe_selected, train_size=0.8, random_state=30027)

X_train_selected = train_selected_df.iloc[:,:-1] # (transformed) attributes
y_train_selected = train_selected_df.iloc[:,-1]  # ground truth labels
X_val_selected = val_selected_df.iloc[:,:-1]
y_val_selected = val_selected_df.iloc[:,-1]

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

# gaussian NB
gnb = GaussianNB()

# linear svm
linearSVM = svm.LinearSVC(random_state=30027, C=1)

# logr
logr = LogisticRegression(solver='sag', max_iter=100)

# decision tree
dt = DecisionTreeClassifier(max_depth = 200, random_state = 30027)

# knn
knn = KNeighborsClassifier(n_neighbors=1)


In [ ]:
from sklearn.ensemble import StackingClassifier
estimators = [('gaussianNB', gnb), ('linearSVM', linearSVM), ('logisticRegression', logr), ('decisionTree', dt), ('kNN', knn)]
stacking = StackingClassifier(estimators = estimators, cv = 5)
stacking.fit(X_train_selected, y_train_selected)

In [ ]:
evaluate(stacking, X_val_selected, y_val_selected)

In [ ]:
stacking.get_params(deep=True)

In [ ]:
param_grid = {'linearSVM__C': range(1,11,5), 'decisionTree__max_depth': range(200, 401, 50), 'kNN__n_neighbors': range(1,6)}
tune_hyperparameter(stacking, param_grid, X_train_selected, y_train_selected, cv = 2)